# EP03: 장애에 강한 멀티 마스터 노드 쉽게 만들기

오프닝: 간단한 본인 소개

Q: 이번에 살펴볼 내용은 어떤 내용인가요?

A: 이번 에피소드에서는 장애에 강한 멀티 마스터 노드 쉽게 만들기라는 주제를 살펴보려고 합니다.

다른 클라우드 컴퓨팅 서비스와 마찬가지로 애저 역시 예기치 않은 장애로부터 자유롭지는 않습니다.

A: 데이터센터가 최상의 상태에서 유지될 수 있도록 마이크로소프트는 항상 최선을 다하지만, 예기치 않은 장애는 늘 대비해야 합니다.

이런 장애를 극복할 수 있도록 마스터 노드를 배치하는 방법을 살펴보겠습니다.

Q: 데이터센터 수준의 장애는 어떻게 방지할 수 있나요?

A: 다른 퍼블릭 클라우드 컴퓨팅 서비스와 마찬가지로 애저에서도 서로 다른 파워 서플라이와 네트워크 회선을 사용하면서도, 낮은 오버헤드와 빠른 응답 속도를 보장하는 분산된 데이터센터 간의 가상 컴퓨터 배포를 지원합니다. 애저에서는 이것은 가상 머신 스케일 셋 방식으로 구현할 수 있습니다.

Q: 그렇다면 가상 머신 스케일 셋을 이용하여 쿠버네티스 클러스터를 배포할 수 있나요?

A: 네. 그렇습니다. 특히 마스터 노드를 가상 머신 스케일 셋으로 배포하면 데이터센터 수준에서 문제가 발생하더라도 장애 상황에 민감하게 반응하지 않는 견고한 쿠버네티스 클러스터를 구축하는데 도움이 됩니다.

지금부터 그 내용을 살펴보도록 하겠습니다.

## 애저 로그인 상태 확인

우선 애저 커맨드 라인 도구의 로그인 상태를 먼저 확인합니다. 로그인 상태가 아니면 다른 터미널에서 `az login` 명령을 실행합니다. 설치되어있지 않다면 OS 버전 별로 애저 커맨드 라인 도구를 설치하고 준비합니다.

In [ ]:
az account list

## AKS 엔진 설치 상태 확인

아래 명령어를 실행하여 `aks-engine`이 설치되었는지 확인합니다. 설치되어있지 않다면 OS 버전 별로 각각 설치 가이드를 따라 설치를 진행합니다.

In [ ]:
aks-engine version

## 서비스 주체 생성

쿠버네티스 클러스터를 만든 후에는 클러스터 수준에서 스스로 애저 리소스들을 제어할 수 있어야 하므로, 서비스 주체 계정을 만들어 계속 해당 계정을 사용하도록 설정합니다.

In [ ]:
$SubscriptionData = (az account list -o json) | ConvertFrom-Json -Depth 16
$SubscriptionId = $SubscriptionData[0].id
$SvcPrincipal = (az ad sp create-for-rbac `
  -n="AzurePlaylist" --role="Contributor" `
  --scopes="/subscriptions/$SubscriptionId" `
  -o json) | ConvertFrom-Json -Depth 16

$PrincipalId = $SvcPrincipal.AppId
$PrincipalPwd = $SvcPrincipal.Password

"Your App ID is $PrincipalId"

## SSH 키 준비하기

아래 명령어를 실행하여 SSH 키의 공개 키를 준비합니다. 설치되어있지 않다면 SSH 키 쌍을 만듭니다.

In [ ]:
$PublicKeyValue = Get-Content -Path "$HOME\.ssh\id_rsa.pub"

## API 모델 준비하기

이제 API 모델을 준비할 차례입니다. AKS 엔진은 코드 기반 인프라 관리 방식을 구현하기 위하여 JSON API 모델을 작성하면, 여기에 맞추어 ARM 템플릿을 만들어줍니다. 이것을 애저 명령줄 도구를 통해 배포하면 손쉽게 쿠버네티스 클러스터를 생성할 수 있습니다.

In [ ]:
$ApiModel = @"
{
  'apiVersion': 'vlabs',
  'properties': {
    'orchestratorProfile': {
      'orchestratorType': 'Kubernetes'
    },
    'servicePrincipalProfile': {
      'clientId': '',
      'secret': ''
    },
    'masterProfile': {
      'count': 3,
      'dnsPrefix': '',
      'vmSize': 'Standard_D2_v3',
      'availabilityProfile': 'VirtualMachineScaleSets'
    },
    'agentPoolProfiles': [
      {
          'name': 'agentpool',
          'count': 3,
          'vmSize': 'Standard_D2_v3',
          'availabilityProfile': 'VirtualMachineScaleSets'
      }
    ],
    'linuxProfile': {
      'adminUsername': 'azureuser',
      'ssh': {
        'publicKeys': [
          {
            'keyData': ''
          }
        ]
      }
    }
  }
}
"@ | ConvertFrom-Json -Depth 16

## API 모델 내용 변경하기

나중에 리눅스 마스터 노드에 접속할 수 있도록 현재 컴퓨터에서 사용하는 SSH 공개 키를 배포할 때 같이 등록합니다. 그리고 서비스 주체를 가리키는 아이디 값과 고유 비밀 키 값을 API 모델에 같이 지정합니다.

In [ ]:
$ApiModel.properties.linuxProfile.ssh.publicKeys[0].keyData = "$PublicKeyValue"
$ApiModel.properties.servicePrincipalProfile.clientId = "$PrincipalId"
$ApiModel.properties.servicePrincipalProfile.secret = "$PrincipalPwd"

In [ ]:
$ApiModel | ConvertTo-Json -Depth 16 | Out-File -Encoding utf8 -Path 'ep03.json' -Force

dir 'ep03*'

## API 모델을 ARM 템플릿으로 변환하고 배포하기

다음의 명령어를 실행하여 API 모델을 ARM 템플릿으로 변환하고 곧바로 애저에 배포하도록 하겠습니다. 이 명령어는 최초에 한 번만 실행하고, 그 이후에는 `ep03_output` 디렉터리 안에 있는 파일들을 이용하여 클러스터를 관리하도록 합니다.

In [ ]:
$DnsPrefix = 'rkttuep03'
$ResourceGroup = 'rkttuep03'
$Location = 'westus2'

aks-engine deploy --dns-prefix "$DnsPrefix" `
    --resource-group "$ResourceGroup" `
    --location "$Location" `
    --api-model "ep03.json" `
    --auto-suffix `
    --output-directory "ep03_output" `
    --auth-method cli

## kubectl 실행해보기

클러스터가 잘 만들어졌는지 확인해보기 위하여 다음과 같이 `KUBECONFIG` 환경 변수를 설정하여 클러스터에 접근하고, 노드와 구성 상태를 확인합니다.

In [ ]:
$env:KUBECONFIG="ep03_output/kubeconfig/kubeconfig.$Location.json"
kubectl get nodes -o wide

## 리소스 그룹 정리

테스트를 위해 생성한 리소스 그룹을 제거하고 모든 리소스를 일괄 정리합니다.

In [ ]:
az group delete --name $ResourceGroup --yes --no-wait

클로징: 이 동영상을 보고 애저를 써봐야겠다고 생각하셨나요? 애저 계정은 누구나 무료로 만들 수 있습니다. 지금 바로 시작해보세요. (가능하다면 할인/무료 쿠폰을 포함한 가입 링크 제공) 다음 에피소드에서는 클라우드 셸을 사용할 수 있는 다양한 방법을 알아보겠습니다.